In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feature_engine.outliers import Winsorizer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from feature_engine.outliers import Winsorizer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn import set_config, get_config
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from pprint import pprint
from collections import defaultdict
import tools
from sklearn.ensemble import GradientBoostingRegressor

set_config(transform_output="pandas")
from mlxtend.evaluate.time_series import GroupTimeSeriesSplit, plot_splits, print_cv_info, print_split_info


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format
import warnings; warnings.filterwarnings('ignore')


In [138]:
old_train = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/train.csv')
new_train = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting_new/revealed_test.csv')

old_test = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/test.csv')
sample_submission = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/sample_submission.csv')

train = pd.concat((old_train, new_train))
test = old_test[~old_test['first_day_of_month'].isin(new_train['first_day_of_month'])]

train['is_test'] = 0 ; test['is_test'] = 1

data = pd.concat((
        train,
        test)
        )\
    .reset_index(drop=True)\
    .assign(
        cfips = lambda df: df['cfips'].astype(str).str.zfill(5),
        date = lambda df: pd.to_datetime(df["first_day_of_month"]),
        mdensity_t0 = lambda df: df['microbusiness_density'],
        active_t0 = lambda df: df['active'],
        )\
    .sort_values(['cfips','date'], ascending=True)\
    .assign(
    
        state_i = lambda df: df['cfips'].apply(lambda x: x[:2]),
        county_i = lambda df: df['cfips'].apply(lambda x: x[2:]),
        
        year = lambda df: df['date'].dt.year,
        date = lambda df: df["date"].dt.date,
        # month = lambda df: df['date'].dt.month,

        dcount = lambda df: df.groupby('cfips')['row_id'].cumcount(),
        
        active_lag1 = lambda df: df.groupby('cfips')['active_t0'].shift(1),
        active_lag2 = lambda df: df.groupby('cfips')['active_t0'].shift(2),
        active_lag3 = lambda df: df.groupby('cfips')['active_t0'].shift(3),
        active_lag4 = lambda df: df.groupby('cfips')['active_t0'].shift(4),
        active_lag5 = lambda df: df.groupby('cfips')['active_t0'].shift(5),
        active_lag6 = lambda df: df.groupby('cfips')['active_t0'].shift(6),
        
        target_0 = lambda df: np.nan_to_num(df['active']),
        target_1 = lambda df: np.nan_to_num(df['active']),
        target_2 = lambda df: np.nan_to_num(df['active']),

    

    )\
    .drop(['county','state'], axis='columns')
# .sort_index(ascending=True)

assert all(data.groupby('cfips')['county_i'].nunique() == 1)
assert all(data.groupby('cfips')['state_i'].nunique() == 1)
assert data['cfips'].nunique() == 3135 # there are 3135 county,state tuples
assert data['dcount'].nunique() == 47 # there are 47 series for each county state tuple
assert data.query('is_test==0')['dcount'].nunique() == 41 # there are 41 series in the train set. 
assert data.query('is_test==1')['dcount'].nunique() == 6  # there are 6 series in the test set. 

#The private leaderboard will include 03-2023, 04-2023, 05-2023
#The public leaderboard includes the first month 11-2022. Probably it will be updated later as 12-2022,01-2023 and 02-2023
#The LB is updated as 01-2023


In [139]:
# adding census data
data_census = []
for year in range(2017,2022):
    COLS = ['GEO_ID','NAME','S0101_C01_026E']
    data_census_i = pd.read_csv(f'./data/raw/census_data_1/ACSST5Y{year}.S0101-Data.csv',usecols=COLS)
    data_census_i = data_census_i.iloc[1:]
    data_census_i['population'] = data_census_i['S0101_C01_026E'].astype('int')


    data_census_i['cfips'] = data_census_i.GEO_ID.apply(lambda x: f"{int(x.split('US')[-1]):05}" )
    data_census_i['year'] = year+2
    data_census.append(data_census_i[['cfips','year','population']])

data_census = pd.concat((data_census),axis='rows')


In [140]:
data = data.merge(data_census, on=['cfips','year'], how='left')

In [141]:
data.head()

,row_id,cfips,first_day_of_month,microbusiness_density,active,is_test,date,mdensity_t0,active_t0,state_i,county_i,year,dcount,active_lag1,active_lag2,active_lag3,active_lag4,active_lag5,active_lag6,target_0,target_1,target_2,population
0,1001_2019-08-01,01001,2019-08-01,3.01,1249.00,0,2019-08-01,3.01,1249.00,01,001,2019,0,NaN,NaN,NaN,NaN,NaN,NaN,1249.00,1249.00,1249.00,41527
1,1001_2019-09-01,01001,2019-09-01,2.88,1198.00,0,2019-09-01,2.88,1198.00,01,001,2019,1,1249.00,NaN,NaN,NaN,NaN,NaN,1198.00,1198.00,1198.00,41527
2,1001_2019-10-01,01001,2019-10-01,3.06,1269.00,0,2019-10-01,3.06,1269.00,01,001,2019,2,1198.00,1249.00,NaN,NaN,NaN,NaN,1269.00,1269.00,1269.00,41527
3,1001_2019-11-01,01001,2019-11-01,2.99,1243.00,0,2019-11-01,2.99,1243.00,01,001,2019,3,1269.00,1198.00,1249.00,NaN,NaN,NaN,1243.00,1243.00,1243.00,41527
4,1001_2019-12-01,01001,2019-12-01,2.99,1243.00,0,2019-12-01,2.99,1243.00,01,001,2019,4,1243.00,1269.00,1198.00,1249.00,NaN,NaN,1243.00,1243.00,1243.00,41527


# Sample Data

In [142]:
# PARAMETERS
n_SPLITS = 5 
n_TRAIN_TRAIN_SIZE = 6
n_TRAIN_PERIOD = n_TRAIN_TRAIN_SIZE + 3 + n_SPLITS - 1 


TEST_DATES = list(np.sort(data.query('is_test==1')['date'].unique())[:3])
TEST_PERIOD = list(np.sort(data.query('is_test==1')['dcount'].unique())[:3])

TRAIN_PERIOD = list(np.sort(data.query('is_test==0')['dcount'].unique())[-n_TRAIN_PERIOD:])
TRAIN_DATES = list(np.sort(data.query('is_test==0')['date'].unique())[-n_TRAIN_PERIOD:])

LEAKAGE = ['mdensity_t0','active_t0']
TARGETS = ['target_0', 'target_1', 'target_2']
FEATURES = ['population']
LAG_TARGET = ['active_lag1', 'active_lag2', 'active_lag3','active_lag4','active_lag5','active_lag6']

In [143]:
# data[data['dcount'].isin(TEST_PERIOD)].head()
# sample = data[data.cfips.isin(['01001'])] # sample = data[data.cfips.isin(['01001','56045'])]
sample = data.copy()
sample.loc[sample.is_test==1,TARGETS]  = np.nan
sample = sample.set_index(['date','cfips']).sort_index().loc[TRAIN_DATES+TEST_DATES]
sample = sample[['dcount','county_i'] + LAG_TARGET + TARGETS + FEATURES+ LEAKAGE]
sample_train= sample.query("dcount in @TRAIN_PERIOD") ; sample_test= sample.query("dcount in @TEST_PERIOD")
train_X = sample_train.drop(TARGETS,axis='columns') ; train_y = sample_train[TARGETS]
test_X = sample_test.drop(TARGETS,axis='columns') ; test_y = sample_test[TARGETS]


In [144]:
train_X

dcount county_i  active_lag1  active_lag2  active_lag3  active_lag4  active_lag5  active_lag6  population  mdensity_t0  active_t0
date       cfips                                                                                                                                   
2021-12-01 01001      28      001      1350.00      1351.00      1344.00      1358.00      1354.00      1359.00       42175         3.29    1386.00
           01003      28      003     13162.00     13048.00     12998.00     13192.00     13301.00     13456.00      166595         7.93   13211.00
           01005      28      005       231.00       228.00       225.00       232.00       230.00       222.00       20054         1.15     231.00
           01007      28      007       220.00       212.00       212.00       216.00       221.00       221.00       17862         1.21     216.00
           01009      28      009       768.00       767.00       766.00       758.00       760.00       759.00       44292         1.75     776.00
...                  ...      ...          ...          ...          ...          ...          ...          ...         ...          ...        ...
2022-12-01 56037      40      037       902.00       905.00       901.00       901.00       909.00       892.00       32049         2.88     922.00
           56039      40      039      5054.00      5035.00      5000.00      4999.00      4971.00      4916.00       19164        26.31    5043.00
           56041      40      041       583.00       582.00       580.00       577.00       578.00       567.00       14516         4.05     588.00
           56043      40      043       190.00       189.00       194.00       194.00       195.00       189.00        6045         3.06     185.00
           56045      40      045       100.00       100.00       100.00       100.00       101.00       101.00        5601         1.80     101.00

[40755 rows x 11 columns]

# Pipelining

In [198]:
lag=1
list_cols_model = [[f'active_lag{lag_i+model_i+1}' for lag_i in range(lag)] for model_i in range(3)]

dic_pipelines = {}
y_test_preds  = [] 

y_val_preds = defaultdict(list)
errors = defaultdict(list)


for model_i in range(3):

    train_y_i = train_y.iloc[:, model_i]
    
    cv_args = {"test_size": 1, "n_splits": n_SPLITS, "train_size": n_TRAIN_TRAIN_SIZE, 'gap_size': 0}
    
    cv = GroupTimeSeriesSplit(**cv_args)

    # new_features = Pipeline([('select', SimpleFeatureEngineering(features=list_cols_model[model_i]))])
    # print(list_cols_model[model_i])

    raw_features = Pipeline([('select', tools.ColumnSelector(features=list_cols_model[model_i]))])
    
    merge_features_numeric = FeatureUnion([
        # ('new_features', new_features),
        ('raw_features', raw_features)
    ])

    final_features_numeric = Pipeline([
                            ('merge_features',merge_features_numeric),
                            # ('remove_outliers', Winsorizer(capping_method='iqr', tail='both',fold=1)),
                            # ('standart_scaler', StandardScaler())
                            ]
                            )

    
    model = TransformedTargetRegressor(regressor=LinearRegression())
    # model = TransformedTargetRegressor(regressor=tools.LagModel(), transformer=None)
    
    
    model_pipeline = Pipeline([
        ("transform", final_features_numeric),
        ("model", model)
    ])

    dic_pipelines[f'pipeline_model_{model_i}'] = model_pipeline
    
    param_grid = {}
    grid = GridSearchCV(dic_pipelines[f'pipeline_model_{model_i}'], scoring=make_scorer(tools.SMAPE_1, greater_is_better=False), param_grid=param_grid, cv=cv)
    grid.fit(train_X, train_y_i, groups=train_X['dcount'])
    
    # print(grid.cv_results_)
    # print(grid.best_estimator_)
    
    ## CHECK

    best_model = grid.best_estimator_

    check_train_period = TRAIN_DATES[-1-n_TRAIN_TRAIN_SIZE: -1] 
    validation_period = TRAIN_DATES[-1] 
    
    best_model.fit(train_X.loc[check_train_period], train_y_i.loc[check_train_period])   
    
    y_val_pred =  best_model.predict(train_X.loc[validation_period])   
    y_val_preds[f'target_{model_i}'] = y_val_pred
    y_val_i = train_y_i.loc[validation_period]    
    errors[f'error_{model_i}'] = tools.SMAPE_1(y_true=y_val_i, y_pred=y_val_pred)

    # INFERENCE
    # final_train_period = TRAIN_DATES[-n_TRAIN_TRAIN_SIZE:] 

    # best_model.fit(train_X.loc[final_train_period], train_y_i.loc[final_train_period])   

    # y_test_pred =  best_model.predict(test_X.loc[TEST_DATES[model_i]] )
    # y_test_preds.append(y_test_pred)

# test_X['active'] = np.concatenate((y_test_preds))

# prepare data for error analysis
val_X = train_X.loc[validation_period]
y_val_preds =  pd.DataFrame(y_val_preds, index=val_X.index)
val_X = pd.concat((val_X, y_val_preds), axis=1)

test_X['microbusiness_density'] = 100*test_X['active']/test_X['population']

(array([1.00938379]), 3.2076890546904906)

In [173]:
# # # Prepare submission file

# date_submission = '0303'
# local_score = round(errors['error_0'],2)
# model_name = 'regression_lag_1_4'

# submission = tools.create_submission(test_X,date_submission, model_name, local_score, sample_submission)

# ERROR ANALYSIS

In [214]:
val_X['error_0']= val_X[['active_t0','target_0']].apply(lambda x: tools.SMAPE_1(x[[0]],x[[1]]),axis=1)
val_X['error_1']= val_X[['active_t0','target_1']].apply(lambda x: tools.SMAPE_1(x[[0]],x[[1]]),axis=1)
val_X['error_2']= val_X[['active_t0','target_2']].apply(lambda x: tools.SMAPE_1(x[[0]],x[[1]]),axis=1)

In [218]:
best_model[-1].regressor_.coef_ , best_model[-1].regressor_.intercept_
1.00938379*100.00+3.2076890546904906

104.14606805469049

In [219]:
errors = val_X.sort_values('error_0', ascending=False)
# errors['cum_error'] = errors['error_0'].expanding().mean()
# errors['cum_population'] = errors['population'].expanding().sum()
# val_X

In [151]:
errors[errors['error_0']>20]


,dcount,county_i,active_lag1,active_lag2,active_lag3,active_lag4,active_lag5,active_lag6,population,mdensity_t0,active_t0,target_0,target_1,target_2,error_0,error_1,error_2,cum_error
cfips,,,,,,,,,,,,,,,,,,
48063,40,063,275.00,272.00,262.00,258.00,261.00,260.00,9540,11.86,1131.00,276.14,274.26,267.65,121.50,121.93,123.45,121.50
28055,40,055,0.00,0.00,0.00,0.00,0.00,0.00,1057,0.09,1.00,0.27,0.79,3.24,116.12,23.96,105.63,118.81
13211,40,211,880.00,885.00,881.00,891.00,884.00,862.00,14584,17.80,2596.00,884.39,893.56,892.93,98.36,97.57,97.63,111.99
29063,40,063,1292.00,1135.00,130.00,128.00,130.00,133.00,9824,4.39,431.00,1255.39,1089.07,134.68,97.77,86.58,104.77,108.44
19065,40,065,257.00,267.00,353.00,353.00,358.00,343.00,15433,4.94,762.00,262.06,274.80,359.64,97.64,93.98,71.75,106.28
38079,40,079,65.00,63.00,63.00,63.00,64.00,62.00,9529,1.60,152.00,65.51,64.31,66.89,79.53,81.08,77.76,101.82
39127,40,127,349.00,350.00,350.00,350.00,354.00,348.00,27582,2.86,788.00,350.96,353.70,356.99,76.74,76.08,75.29,98.24
48205,40,205,11.00,11.00,11.00,11.00,11.00,11.00,4388,0.48,21.00,11.32,11.88,14.35,59.92,55.49,37.60,93.45
21189,40,189,9.00,9.00,8.00,9.00,10.00,10.00,3571,0.48,17.00,9.26,9.86,11.58,58.94,53.16,37.96,89.61


In [152]:
errors

,dcount,county_i,active_lag1,active_lag2,active_lag3,active_lag4,active_lag5,active_lag6,population,mdensity_t0,active_t0,target_0,target_1,target_2,error_0,error_1,error_2,cum_error
cfips,,,,,,,,,,,,,,,,,,
48063,40,063,275.00,272.00,262.00,258.00,261.00,260.00,9540,11.86,1131.00,276.14,274.26,267.65,121.50,121.93,123.45,121.50
28055,40,055,0.00,0.00,0.00,0.00,0.00,0.00,1057,0.09,1.00,0.27,0.79,3.24,116.12,23.96,105.63,118.81
13211,40,211,880.00,885.00,881.00,891.00,884.00,862.00,14584,17.80,2596.00,884.39,893.56,892.93,98.36,97.57,97.63,111.99
29063,40,063,1292.00,1135.00,130.00,128.00,130.00,133.00,9824,4.39,431.00,1255.39,1089.07,134.68,97.77,86.58,104.77,108.44
19065,40,065,257.00,267.00,353.00,353.00,358.00,343.00,15433,4.94,762.00,262.06,274.80,359.64,97.64,93.98,71.75,106.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29203,40,203,128.00,132.00,132.00,133.00,137.00,135.00,6436,2.00,129.00,129.00,133.93,137.08,0.00,3.75,6.08,1.90
28081,40,081,2778.00,2776.00,2759.00,2742.00,2723.00,2670.00,63723,4.38,2791.00,2790.97,2798.11,2783.56,0.00,0.25,0.27,1.90
47041,40,041,430.00,430.00,423.00,429.00,435.00,426.00,15725,2.75,432.00,432.00,434.32,431.68,0.00,0.54,0.07,1.90


In [153]:
errors['c_population']=  pd.cut(np.log1p(errors['population']),5)
errors['c_population'].value_counts()
errors.groupby(['c_population'])['error_0'].describe()

,count,mean,std,min,25%,50%,75%,max
c_population,,,,,,,,
"(4.51, 6.793]",40.00,7.47,9.41,0.54,2.02,3.72,10.40,49.44
"(6.793, 9.064]",752.00,3.08,6.74,0.00,0.77,1.47,2.99,116.12
"(9.064, 11.335]",1782.00,1.63,6.09,0.00,0.31,0.72,1.43,121.50
"(11.335, 13.606]",517.00,0.78,2.08,0.00,0.18,0.39,0.69,37.89
"(13.606, 15.878]",44.00,0.50,0.48,0.02,0.18,0.39,0.60,2.07


In [154]:
errors = errors[errors['error_0']>1]

In [155]:
# plt.scatter( np.log1p(errors['population']), np.log1p(errors['error_0']))